# visualization and imports

In [1]:
import os
import numpy as np
import pandas as pd
import sklearn

In [2]:
## making directory to unzip dataset we have downloaded from kaggle
os.makedirs('/content/spaceship', exist_ok = True)
!unzip /content/spaceship-titanic.zip -d /content/spaceship

Archive:  /content/spaceship-titanic.zip
  inflating: /content/spaceship/sample_submission.csv  
  inflating: /content/spaceship/test.csv  
  inflating: /content/spaceship/train.csv  


In [3]:
## build dataframes
sample_submission = pd.read_csv('/content/spaceship/sample_submission.csv')
test = pd.read_csv('/content/spaceship/test.csv')
train = pd.read_csv('/content/spaceship/train.csv',index_col = 'PassengerId')
display(train.head())
## train datas
x_train = train.drop(['Transported'],axis = 1)
y_train = train[['Transported']]
## test datas
x_test = test
display(x_train.head(),y_train.head())
print('the amount of nan values in each column',train.isnull().sum(),sep = '\n')


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
PassengerId,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


,Transported
PassengerId,
0001_01,False
0002_01,True
0003_01,False
0003_02,False
0004_01,True


the amount of nan values in each column
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64


preprocessing data and removing columns with high cardinality

In [4]:
obj_cols = list(train.select_dtypes(include = 'object').columns)
numerical_cols = list(x_train.select_dtypes(include = np.number))
for col in obj_cols:
  print(f' for {col} we have', x_train[col].nunique(),'number of unique values')

low_cardi = [col for col in obj_cols if len(set(train[col])) <10]
x_train_low = x_train[low_cardi]
x_test_low = x_test[low_cardi]
x_numeric = x_train.select_dtypes(np.number)
x_test_numeric = x_test.select_dtypes(np.number)
y_train = y_train.replace([False,True],[0,1])
display(x_train_low.head(),y_train.head())

 for HomePlanet we have 3 number of unique values
 for CryoSleep we have 2 number of unique values
 for Cabin we have 6560 number of unique values
 for Destination we have 3 number of unique values
 for VIP we have 2 number of unique values
 for Name we have 8473 number of unique values


,HomePlanet,CryoSleep,Destination,VIP
PassengerId,,,,
0001_01,Europa,False,TRAPPIST-1e,False
0002_01,Earth,False,TRAPPIST-1e,False
0003_01,Europa,False,TRAPPIST-1e,True
0003_02,Europa,False,TRAPPIST-1e,False
0004_01,Earth,False,TRAPPIST-1e,False


,Transported
PassengerId,
0001_01,0
0002_01,1
0003_01,0
0003_02,0
0004_01,1


# without Pipelines

## Imputation

In [5]:
##checking most frequent values in each column
dict_freq = {}
for i in x_train_low.columns:
  dict_freq[i] = set(x_train_low[i].value_counts().index[:3])
print(dict_freq)

{'HomePlanet': {'Mars', 'Europa', 'Earth'}, 'CryoSleep': {False, True}, 'Destination': {'PSO J318.5-22', 'TRAPPIST-1e', '55 Cancri e'}, 'VIP': {False, True}}


In [6]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy = 'most_frequent')
x_categ_imp = imp.fit_transform(x_train_low)
x_categ_imp = pd.DataFrame(x_categ_imp,columns = x_train_low.columns , index = x_train.index)
x_numeric_imp = imp.fit_transform(x_train.select_dtypes(np.number))
x_numeric_imp = pd.DataFrame(x_numeric_imp,columns = x_numeric.columns , index = x_numeric.index)
# display(x_categ_imp,x_numeric_imp)
## checking imputation work correctly
print('number of missing values in our categorical dataset\n',x_categ_imp.isnull().sum(),'\n'*2 ,
      'and number of missing values in our numerical dataset\n',x_numeric_imp.isnull().sum(),'\n'*2,sep = '')
print('befor imputation\n',x_train['HomePlanet'].value_counts(),'\n\nafter imputation\n',x_categ_imp['HomePlanet'].value_counts(),sep = '')

number of missing values in our categorical dataset
HomePlanet     0
CryoSleep      0
Destination    0
VIP            0
dtype: int64

and number of missing values in our numerical dataset
Age             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64


befor imputation
Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

after imputation
Earth     4803
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64


## one hot encoding

In [7]:
from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder(handle_unknown = 'ignore',sparse_output = False)
x_onehot = one_hot.fit_transform(x_categ_imp)
x_onehot = pd.DataFrame(x_onehot,index = x_categ_imp.index)
x_train_processed = pd.concat([x_onehot,x_numeric_imp], axis = 1)
display(x_train_processed)

,0,1,2,3,4,5,6,7,8,9,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
PassengerId,,,,,,,,,,,,,,,,
0001_01,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0
0002_01,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,24.0,109.0,9.0,25.0,549.0,44.0
0003_01,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,58.0,43.0,3576.0,0.0,6715.0,49.0
0003_02,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,33.0,0.0,1283.0,371.0,3329.0,193.0
0004_01,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,16.0,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,41.0,0.0,6819.0,0.0,1643.0,74.0
9278_01,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0
9279_01,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,26.0,0.0,0.0,1872.0,1.0,0.0


## random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
x_train_processed.columns = x_train_processed.columns.astype(dtype='str')
clf = RandomForestClassifier()
metric = 'accuracy'
cross_val = cross_val_score(clf,x_train_processed,np.array(y_train).reshape([len(y_train),]),cv = 5 , scoring = metric)
print(f'cross validation score with{metric} is : {cross_val} and its mean is {cross_val.mean()}')
# clf.predict(x_test)
# clf.fit(x_train_processed,y_train)
# clf.predict(x_train_processed)

cross validation score withaccuracy is : [0.78550891 0.78378378 0.78148361 0.78365938 0.79286536] and its mean is 0.785460209860964


until now we use usual coding for preprocessing the train datas so we need to repeat it for test, so decided to do all the codes above with help of pipelines to make code more readable and shorter and also avoid forgetting any datapreprocessing stage for test data

#with Pipeline

## without normalize

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
def model_pipeline(choosen_model = RandomForestClassifier()):
  numerical_imp = SimpleImputer(strategy = 'median')
  categorical_pipeline = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'most_frequent')),
                                           ('onehot',one_hot)])
  transforms = ColumnTransformer([('categorical', categorical_pipeline,low_cardi ),
                                  ('numerical', numerical_imp, numerical_cols)])
  model = choosen_model

  final_pipeline = Pipeline(steps = [('preprocessing', transforms),
                                     ('model', model)])
  return final_pipeline
# transforms.fit_transform(x_train)
numerical_set = []
dic_of_models = {1:RandomForestClassifier() , 2:LogisticRegression() , 3:SVC() , 4:KNeighborsClassifier() }
quit = False
while not quit:
  inp = input('''choose one model to evaluate its cross validation:
  1- RandomForestClassifier
  2- logistic regression
  3- support vector machine
  4- K nearest neighbor
  ''')
  inp = int(inp)
  numerical_set.append(inp)
  quit = int(input('''' do you wanna quit? 0:no 1:yes
  '''))
# cros_val = cross_val_score(model_pipeline(),x_train,np.array(y_train).reshape((len(y_train),)))
our_models = []
for i in numerical_set:
  our_models.append(dic_of_models.get(i,dic_of_models[1]))


cros_val_map = map(lambda iterative_model: cross_val_score(model_pipeline(choosen_model = iterative_model),x_train,np.array(y_train).reshape((len(y_train),))),our_models)
cros_val = np.array(list(cros_val_map))
for i in range(len(our_models)):
  print(f' our cross validation set for{our_models[i]}  is: {cros_val[i,:]} and its mean is: {cros_val[i,:].mean()}')

choose one model to evaluate its cross validation:
  1- RandomForestClassifier
  2- logistic regression
  3- support vector machine
  4- K nearest neighbor
  1
' do you wanna quit? 0:no 1:yes
  0
choose one model to evaluate its cross validation:
  1- RandomForestClassifier
  2- logistic regression
  3- support vector machine
  4- K nearest neighbor
  2
' do you wanna quit? 0:no 1:yes
  0
choose one model to evaluate its cross validation:
  1- RandomForestClassifier
  2- logistic regression
  3- support vector machine
  4- K nearest neighbor
  3
' do you wanna quit? 0:no 1:yes
  0
choose one model to evaluate its cross validation:
  1- RandomForestClassifier
  2- logistic regression
  3- support vector machine
  4- K nearest neighbor
  4
' do you wanna quit? 0:no 1:yes
  1


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

 our cross validation set forRandomForestClassifier()  is: [0.78320874 0.78378378 0.77975848 0.78193326 0.79401611] and its mean is: 0.7845400746828164
 our cross validation set forLogisticRegression()  is: [0.78090857 0.78320874 0.78033353 0.77963176 0.79459148] and its mean is: 0.7837348157843713
 our cross validation set forSVC()  is: [0.76998275 0.77228292 0.78953422 0.80034522 0.79919448] and its mean is: 0.7862679171593797
 our cross validation set forKNeighborsClassifier()  is: [0.76940771 0.77400805 0.7492812  0.76467204 0.78193326] and its mean is: 0.7678604491424312


results shows that the best alghorithm is random forest

## with normlize

In [52]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
def model_pipeline2( choosen_model = RandomForestClassifier() , fit = True):
  numerical_imp = SimpleImputer(strategy = 'median')
  numerical_norm = StandardScaler()
  numerical_pipeline = Pipeline(steps = [('numerical_imp' , numerical_imp),
                                     ('numerical_norm' , numerical_norm)])
  categorical_pipeline = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'most_frequent')),
                                           ('onehot',one_hot)])
  transforms = ColumnTransformer([('categorical', categorical_pipeline,low_cardi ),
                                  ('numerical', numerical_pipeline, numerical_cols)])

  model = choosen_model
  # display(transforms.transform(x_train))
  final_pipeline = Pipeline(steps = [('preprocessing', transforms),
                                     ('model', model)])
  # display(final_pipeline.fit(x,y))
  # if fit:
  #   return final_pipeline.fit(x,y)
  # else:
  return final_pipeline
# transforms.fit_transform(x_train)

cros_val_nor = cross_val_score(model_pipeline(),x_train,np.array(y_train).reshape([len(y_train),]))
print(f'cross validate score for random forest with normalize data is {cros_val_nor} and its mean is {cros_val_nor.mean()}')
rf_model = model_pipeline()
rf_model.fit(x_train,np.array(y_train).reshape([len(y_train),]))
y_hat = rf_model.predict(x_train)
acc = accuracy_score(y_hat,y_train)
print('accuracy for normalize data after trainig on all training data is',acc)

cross validate score for random forest with normalize data is [0.78378378 0.78148361 0.7786084  0.78883774 0.79746835] and its mean is 0.7860363779297256
accuracy for normalize data after trainig on all training data is 0.9293684573795008


##predict on our test data set and prepare our CSV file

In [79]:
test_pred = pd.DataFrame(rf_model.predict(x_test), index = x_test['PassengerId'],columns = ['Transported'])
# change 0 and 1 to False and True
test_pred = test_pred.astype('bool')
display(test_pred)
test_pred.to_csv('danial_submission.csv')

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,False
...,...
9266_02,True
9269_01,False
9271_01,True
